In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path

import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper

from scipy import spatial, stats
from scipy.ndimage import gaussian_filter
from scipy.integrate import odeint

from genpen.grower import *

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 6421
thetas = np.linspace(0, np.pi*24, n_lines)
radii = np.linspace(0.8, 18, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 3.7
t = np.linspace(0, t_max, 41)
a = 0.1
b = 0.95
c = - 0.02
d = -0.02

all_polys = Polygon()

break_dist = 0.05

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
lbs0 = sa.rotate(lbs0, 90)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0)
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam(plot_id='20210607-185734_004816-7a540-45c64f')

In [ ]:
ac.init_cam()

In [ ]:
ac.grab_frames(wait_time=3)

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 70

In [ ]:
ac.ad.options.accel = 80
ac.ad.options.pen_raise_rate = 85

In [ ]:
ac.ad.options.speed_pendown = 40
ac.ad.update()

In [ ]:
1660 + 706

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=1660)

# circle pack

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox, rad_seq_start=0.5,
    rad_seq_end=0.1, min_allowed_rad=5, n_rads=10, )
trcp.run(1)
trcp.run(25)

In [ ]:
cbounds = sa.scale(trcp.boundary, xfact=0.1, yfact=0.1)

In [ ]:
cc = cbounds.centroid

In [ ]:
cbounds = sa.translate(cbounds, xoff=-cc.x, yoff=-cc.y)

In [ ]:
cbounds.bounds

In [ ]:
n_lines = 4421
pts = []
for d in tqdm(np.linspace(1, 0, n_lines)):
    pts.append(cbounds.interpolate(d, normalized=True))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 4.1
t = np.linspace(0, t_max, 41)
a = 0.1
b = 0.95
c = - 0.02
d = -0.02

all_polys = Polygon()

break_dist = 0.05

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.1])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0[:])
sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.n_layers

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 70

In [ ]:
ac.ad.options.accel = 80
ac.ad.options.pen_raise_rate = 85

In [ ]:
ac.ad.options.speed_pendown = 40
ac.ad.update()

In [ ]:
ac.ad.options.layer = 606
ac.ad.plot_run()

In [ ]:
ac.plot_layers(wait_times=0.)

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 2921
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(0.8, 16, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 12.7
t = np.linspace(0, t_max, 61)
a = 0.1
b = 0.95
c = - 0.02
d = -0.02

all_polys = Polygon()

break_dist = 0.04

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
lbs0 = sa.rotate(lbs0, 90)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0)
# sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
blbs = lbs0.buffer(0.2)

In [ ]:
blbs2 = lbs0.buffer(10).buffer(-9.5)

In [ ]:
not_filled = blbs2.difference(blbs)

In [ ]:
nfp = gp.Poly(not_filled)

In [ ]:
xjitter_func=ss.norm(loc=0, scale=0.4).rvs
yjitter_func=ss.norm(loc=0, scale=1.4).rvs
bhfs = gp.BezierHatchFill(not_filled, xjitter_func=xjitter_func, yjitter_func=yjitter_func, spacing=0.4, fill_inscribe_buffer=2.01)

In [ ]:
fills = bhfs.fill

In [ ]:
fills = [l for l in fills if l.length>0.5]

In [ ]:
sns.displot([np.log10(l.length) for l in fills])

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in fills:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 30
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

# thing

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 1321
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(0.5, 16.5, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 22.7
t = np.linspace(0, t_max, 121)
a = 0.1
b = 0.95
c = - 0.15
d = -0.1

all_polys = Polygon()

break_dist = 0.1

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0)
# sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 30
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

# thing

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 1521
thetas = np.linspace(0, np.pi*10, n_lines)
radii = np.linspace(0.5, 20.5, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v * u) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 22.7
t = np.linspace(0, t_max, 121)
a = 0.1
b = 0.95
c = - 0.15
d = -0.1

all_polys = Polygon()

break_dist = 0.25

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0)
# sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 6
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=20
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
circ = drawbox.centroid.buffer(55)

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.02).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=3,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.2)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.3, 0.8),
            degrees=np.random.uniform(-90,90),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in layer:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 50
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

# thing

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=30
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
center = drawbox.centroid

In [ ]:
n_lines = 521
thetas = np.linspace(0, np.pi*2, n_lines)
radii = np.linspace(14.5, 14.5, n_lines)

In [ ]:
pts = []
for theta, radius in zip(thetas, radii):
    x = np.cos(theta) * radius - 0
    y = np.sin(theta) * radius + 0.
    pts.append(Point(x, y))

In [ ]:
def ode(y, t, a, b, c, d):
    v, u = y
    dvdt = np.sin(b * u) + v * c
    dudt = np.cos(a * v) + u  * d
    dydt = [dvdt, dudt]
    return dydt

In [ ]:


t_max = 22.7
t = np.linspace(0, t_max, 221)
a = 0.4
b = 0.3
c = -0.2
d = -0.1

all_polys = Polygon()

break_dist = 0.7

lines = []
lfs = MultiLineString()
allowed_counter = 0
for ii, pt in enumerate(tqdm(pts)):
    sol = odeint(ode, [pt.x, pt.y], t, args=(a, b, c, d))
    mpt = MultiPoint(sol)
    if ii == 0:
        ls = LineString(mpt)
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
    else:
        allowed_counter = 0
        for _pt in mpt:
            dist = _pt.distance(lfs)
            if dist < break_dist:
                break
            allowed_counter += 1
    if allowed_counter > 1:
        ls = LineString(mpt[:allowed_counter])
        lfs = gp.merge_LineStrings([lfs, ls])
        lines.append(ls)
        
lbs0 = gp.merge_LineStrings([l for l in lines if l.length > 0.9])     
lbs0 = gp.make_like(gp.merge_LineStrings(lbs0), drawbox)



In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

sk.geometry(lbs0)
# sk.vpype('linesimplify')

sk.display(color_mode='layer')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in lbs0:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 26
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=19)

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=40
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
circ = drawbox.centroid.buffer(115)

In [ ]:
xgen = ss.uniform(loc=0.5, scale=0.02).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=0,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(2.2, 4.2)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(2.3, 3.8),
            degrees=np.random.choice([-45, 45]),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in layer:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 50
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=76)

# heart of everything

In [ ]:
# make page
paper_size = '11x14 inches'
border:float=50
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
circ = drawbox.centroid.buffer(55)

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.02).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=1,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

inds = np.arange(len(bps))
split_bps = np.array_split(bps, n_layers)

layers = []
for ii in range(n_layers):
    this_bps = split_bps[ii]
    fills = []
    for p in this_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 3.7)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.6, 0.8),
            degrees=np.random.uniform(30, 35),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=100,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in layer:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
import fn

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
from genpen.axicam import AxiCam

In [ ]:
ac = AxiCam()

In [ ]:
ac.init_cam()

In [ ]:
ac.toggle_pen()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 30
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

In [ ]:
circ = drawbox.centroid.buffer(45)
xgen = ss.uniform(loc=0.35, scale=0.02).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    circ, 
    split_func=split_func,
    p_continue=0.85, 
    depth=0, 
    depth_limit=6,
    buffer_frac=-0.0
)


bps = MultiPolygon([p for p in splits])

In [ ]:
keep_bps = np.random.choice(bps, int(len(bps)*0.6))

In [ ]:
circs = gp.circle_pack_within_poly(poly=circ, rads=np.linspace(3, 1, 5))


In [ ]:
middle_circs = circs.intersection(bps.buffer(-2))
middle_circs

In [ ]:
len(middle_circs.boundary)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')

counter = 0
for ls in middle_circs.boundary:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
ac = AxiCam()

ac.init_cam()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 30
ac.ad.update()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=0)

In [ ]:
gs = []

In [ ]:
init_poly = circ

In [ ]:
params = GrowerParams(
#     rad_func='loss_scaled_rad',
#     rad_range=(4, 1),
    rads=ss.uniform(loc=3, scale=6).rvs,
#     rads=1,
    n_pts_eval_per_iter=4,
    n_pts_add_per_iter=2,
    pt_to_poly_func='reg_poly',
    loss_func='negative_distance_from_target',
    rotations=ss.uniform(loc=0, scale=360).rvs,
    loss_range=(0, 7),
    halt_condition_func='below_loss_threshold',
    n_corners=6,
    loss_threshold=-120
)
g = Grower(poly=init_poly, params=params, target=drawbox.centroid)
g.grow(130)
init_poly = g.agg_poly
gs.append(g)

In [ ]:
g.mpoly

In [ ]:
polys = g.polys
_ = polys.pop(0)

In [ ]:
target = Point(30, 300)

In [ ]:
ds = []
for poly in polys:
    d = poly.distance(target)
    ds.append(d)

In [ ]:
fills = []
for poly in polys:
    d = poly.distance(target)
    prms = gp.ScaleTransPrms(
        n_iters=200,
        d_buffer=np.interp(d, (50, 250), (-0.9, -0.25)),
        d_translate_factor=0.6,
        angles=np.pi * 3/4,
    )
    p = gp.Poly(poly)
    p.fill_scale_trans(**prms.prms)
    fills.append(p.fill)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.centered = False

counter = 0

for ls in fills:
    counter += 1
    sk.stroke(counter)
    sk.geometry(ls)
    

sk.display(color_mode='layer')

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [ ]:
ac = AxiCam()

ac.init_cam()

In [ ]:
ac.cam.video_preview()

In [ ]:
ac.cam.close()

In [ ]:
ac.ad.options.speed_penup = 90

In [ ]:
ac.ad.options.accel = 60
ac.ad.options.pen_raise_rate = 95

In [ ]:
ac.ad.options.speed_pendown = 60
ac.ad.update()

In [ ]:
ac.cam.get_video()

In [ ]:
ac.plot_layers(wait_times=0.3, start_layer=126)